In [1]:
# 1.  الإعدادات العامة
# ==========================
import pandas as pd, numpy as np, datetime as dt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout, Input
from sklearn.metrics import average_precision_score, recall_score, precision_score

2025-11-14 23:58:03.519382: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-14 23:58:03.574217: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-14 23:58:07.293041: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-14 23:58:07.293041: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [5]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, recall_score, precision_score, precision_recall_curve
from sklearn.utils.class_weight import compute_class_weight
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
import pickle

np.random.seed(42)
torch.manual_seed(42)

cities = ['Marrakech', 'Casablanca', 'Ouarzazate', 'Rabat', 'Tangier', 'Agadir']
city_base = {'Marrakech': (6.8, 36), 'Casablanca': (12.3, 28), 'Ouarzazate': (2.1, 38),
             'Rabat': (10.5, 25), 'Tangier': (11.0, 24), 'Agadir': (8.2, 30)}

all_dfs = []
normalizers = {}
for city in cities:
    dates = pd.date_range("2020-01-01", "2025-11-15", freq="D")
    n = len(dates)
    base_rad, base_temp = city_base[city]
    rad = np.zeros(n)
    temp = np.zeros(n)
    rad[0] = base_rad + np.random.normal(0, 0.4)
    temp[0] = base_temp + np.random.normal(0, 2)
    for i in range(1, n):
        rad[i] = 0.7 * rad[i-1] + 0.3 * base_rad + np.random.normal(0, 0.2)  # Persistence
        temp[i] = 0.7 * temp[i-1] + 0.3 * base_temp + np.random.normal(0, 1)  # Persistence
    # Add seasonal trends
    day_num = np.arange(n)
    seasonal_temp = 5 * np.sin(2 * np.pi * day_num / 365)  # Annual cycle
    seasonal_rad = 1.5 * np.sin(2 * np.pi * day_num / 365 + np.pi)  # Phase offset
    temp += seasonal_temp
    rad += seasonal_rad
    df = pd.DataFrame({'date': dates, 'city': city, 'mean_rad': rad, 'temp_max': temp})
    df['rad_rolling'] = df['mean_rad'].rolling(window=3, min_periods=1).mean()
    df['temp_rolling'] = df['temp_max'].rolling(window=3, min_periods=1).mean()  # New rolling temp
    prob = 0.05 + 0.30 * (rad < base_rad - 0.3) + 0.25 * (temp > base_temp + 2) + \
           0.25 * (df['rad_rolling'] < base_rad - 0.2) + 0.20 * (df['temp_rolling'] > base_temp + 1.5)
    prob = np.clip(prob, 0, 1)
    label = np.random.binomial(1, prob, n)
    df['outage_label'] = label
    # Normalization
    df['mean_rad_norm'] = (df['mean_rad'] - df['mean_rad'].mean()) / df['mean_rad'].std()
    df['temp_norm'] = (df['temp_max'] - df['temp_max'].mean()) / df['temp_max'].std()
    df['rad_rolling_norm'] = (df['rad_rolling'] - df['rad_rolling'].mean()) / df['rad_rolling'].std()
    df['temp_rolling_norm'] = (df['temp_rolling'] - df['temp_rolling'].mean()) / df['temp_rolling'].std()
    all_dfs.append(df)
    # Save normalizers
    normalizers[city] = {
        'mean_rad': (df['mean_rad'].mean(), df['mean_rad'].std()),
        'temp_max': (df['temp_max'].mean(), df['temp_max'].std()),
        'rad_rolling': (df['rad_rolling'].mean(), df['rad_rolling'].std()),
        'temp_rolling': (df['temp_rolling'].mean(), df['temp_rolling'].std())
    }
all_df = pd.concat(all_dfs, ignore_index=True)

# Save normalizers for future predictions
with open("normalizers.pkl", 'wb') as f:
    pickle.dump(normalizers, f)

# One-hot city
all_df = pd.get_dummies(all_df, columns=['city'], dtype=float)

features = ['mean_rad_norm', 'temp_norm', 'rad_rolling_norm', 'temp_rolling_norm'] + [col for col in all_df.columns if col.startswith('city_')]
seq_len = 7
X, y, indices = [], [], []
city_cols = [col for col in all_df.columns if col.startswith('city_')]
for _, group in all_df.groupby(city_cols, sort=False):
    group_features = group[features]
    group_label = group['outage_label']
    group_indices = group.index
    for i in range(seq_len, len(group)):
        X.append(group_features.iloc[i - seq_len:i].values)
        y.append(group_label.iloc[i])
        indices.append(group_indices[i])
X, y = np.array(X), np.array(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train_t = torch.from_numpy(X_train).float()
y_train_t = torch.from_numpy(y_train).float().unsqueeze(1)
X_val_t = torch.from_numpy(X_val).float()

train_ds = TensorDataset(X_train_t, y_train_t)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)  # Larger batch

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=64):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=3, bidirectional=True, batch_first=True)  # Deeper
        self.dropout = nn.Dropout(0.3)
        self.dense1 = nn.Linear(hidden_size * 2, 16)
        self.dense2 = nn.Linear(16, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.dropout(out)
        out = nn.ReLU()(self.dense1(out))
        out = self.dense2(out)
        return out

model = LSTMModel(len(features))
optimizer = Adam(model.parameters(), lr=0.0005)  # Lower LR
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
pos_weight = torch.tensor([class_weights[1] / class_weights[0]]).float()
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
device = torch.device('cpu')
model.to(device)

best_pr_auc = 0.0
best_model_state = None
patience = 20  # Higher patience
counter = 0
for epoch in range(150):
    model.train()
    for x_b, y_b in train_loader:
        x_b, y_b = x_b.to(device), y_b.to(device)
        optimizer.zero_grad()
        outputs = model(x_b)
        loss = loss_fn(outputs, y_b)
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_t.to(device))
        prob_val = torch.sigmoid(val_outputs).cpu().numpy()[:, 0]
    current_pr_auc = average_precision_score(y_val, prob_val)
    if current_pr_auc > best_pr_auc:
        best_pr_auc = current_pr_auc
        best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f'Early stopping at epoch {epoch}')
            break

if best_model_state is not None:
    model.load_state_dict(best_model_state)

model.eval()
with torch.no_grad():
    prob_val = torch.sigmoid(model(X_val_t.to(device))).cpu().numpy()[:, 0]

precision, recall, thresholds = precision_recall_curve(y_val, prob_val)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)
best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx]
recall_opt = recall_score(y_val, prob_val > best_threshold)
precision_opt = precision_score(y_val, prob_val > best_threshold, zero_division=0)
pr_auc = average_precision_score(y_val, prob_val)
print(f"PR-AUC = {pr_auc:.3f} | Recall (opt @ {best_threshold:.3f}) = {recall_opt:.3f} | Precision (opt) = {precision_opt:.3f}")

with torch.no_grad():
    X_t = torch.from_numpy(X).float().to(device)
    pred_probs = torch.sigmoid(model(X_t)).cpu().numpy()[:, 0]

all_df['pred_prob'] = np.nan
all_df.loc[indices, 'pred_prob'] = pred_probs
all_df.to_csv("pred_result_v3.csv", index=False)
print("Sample:")
print(all_df[['date', 'outage_label', 'pred_prob']].sample(10))
torch.save(model.state_dict(), "model_v3.pt")
print("✅ model_v3.pt saved")

Early stopping at epoch 25
PR-AUC = 0.968 | Recall (opt @ 0.527) = 0.903 | Precision (opt) = 0.925
Sample:
            date  outage_label  pred_prob
9680  2023-01-01             0   0.120054
5969  2024-08-04             0   0.101881
10056 2024-01-12             1   0.619785
2247  2020-04-11             1   0.998503
4807  2021-05-30             1   0.943688
7945  2024-02-16             1   0.988492
7790  2023-09-14             0   0.058838
3007  2022-05-11             1   0.990654
2438  2020-10-19             0   0.060644
5577  2023-07-09             0   0.110914
✅ model_v3.pt saved
Sample:
            date  outage_label  pred_prob
9680  2023-01-01             0   0.120054
5969  2024-08-04             0   0.101881
10056 2024-01-12             1   0.619785
2247  2020-04-11             1   0.998503
4807  2021-05-30             1   0.943688
7945  2024-02-16             1   0.988492
7790  2023-09-14             0   0.058838
3007  2022-05-11             1   0.990654
2438  2020-10-19         

In [8]:
import pandas as pd
import numpy as np
import torch
import pickle
from torch import nn

# 1. تحميل الموديل والمعاير
class LSTMModel(nn.Module):
    def __init__(self, input_size=10):
        super().__init__()
        self.lstm = nn.LSTM(input_size, 64, num_layers=3, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.dense1 = nn.Linear(64 * 2, 16)
        self.dense2 = nn.Linear(16, 1)
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.dropout(out)
        out = nn.ReLU()(self.dense1(out))
        out = self.dense2(out)
        return out

model = LSTMModel(input_size=10)
model.load_state_dict(torch.load("model_v3.pt", map_location='cpu'))
model.eval()

with open("normalizers.pkl", 'rb') as f:
    normalizers = pickle.load(f)

# 2. تحميل البيانات واستعادة عمود city
df = pd.read_csv("pred_result_v3.csv")

# استعادة اسم المدينة من one-hot
city_cols = [c for c in df.columns if c.startswith('city_')]
df['city'] = df[city_cols].idxmax(axis=1).str.replace('city_', '')

# 3. اختيار المدينة
city = "Marrakech"  # غيّر حسب الحاجة
base_rad, base_temp = 6.8, 36

# مثال: بيانات اليوم من API
mean_rad_today = 5.1   # من API
temp_max_today = 38.5

# استخدم آخر 6 أيام من CSV + اليوم من API
last_6 = df[df['city'] == city].tail(6).copy()
today_row = pd.Series({
    'mean_rad': mean_rad_today,
    'temp_max': temp_max_today,
    'city': city
})
today_row['rad_rolling'] = pd.concat([last_6['mean_rad'], pd.Series([mean_rad_today])]).rolling(3).mean().iloc[-1]
today_row['temp_rolling'] = pd.concat([last_6['temp_max'], pd.Series([temp_max_today])]).rolling(3).mean().iloc[-1]

last_7 = pd.concat([last_6, today_row.to_frame().T], ignore_index=True)

# 5. إضافة rolling إذا لم تكن موجودة
if 'rad_rolling' not in last_7.columns:
    last_7['rad_rolling'] = last_7['mean_rad'].rolling(window=3, min_periods=1).mean()
if 'temp_rolling' not in last_7.columns:
    last_7['temp_rolling'] = last_7['temp_max'].rolling(window=3, min_periods=1).mean()

# 6. تطبيع الميزات
def normalize(val, city, feat):
    mean, std = normalizers[city][feat]
    return (val - mean) / std

features = []
for _, row in last_7.iterrows():
    rad_norm = normalize(row['mean_rad'], city, 'mean_rad')
    temp_norm = normalize(row['temp_max'], city, 'temp_max')
    rad_roll_norm = normalize(row['rad_rolling'], city, 'rad_rolling')
    temp_roll_norm = normalize(row['temp_rolling'], city, 'temp_rolling')
    
    # one-hot للمدينة
    city_onehot = [1.0 if c == f"city_{city}" else 0.0 for c in [f"city_{cc}" for cc in normalizers.keys()]]
    
    features.append([rad_norm, temp_norm, rad_roll_norm, temp_roll_norm] + city_onehot)

X_new = torch.tensor([features], dtype=torch.float32)

# 7. التنبؤ
with torch.no_grad():
    prob = torch.sigmoid(model(X_new)).item()

print(f"احتمال انقطاع الكهرباء في {city} غدًا: {prob:.1%}")
if prob > 0.53:
    print("تحذير: احتمال انقطاع مرتفع!")
else:
    print("الوضع مستقر.")

احتمال انقطاع الكهرباء في Marrakech غدًا: 11.2%
الوضع مستقر.


In [17]:
import pandas as pd
import numpy as np
import torch
import pickle
from torch import nn
from datetime import datetime, timedelta

# === 1. تعريف الكلاس (لازم!) ===
class LSTMModel(nn.Module):
    def __init__(self, input_size=10):
        super().__init__()
        self.lstm = nn.LSTM(input_size, 64, num_layers=3, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.dense1 = nn.Linear(128, 16)
        self.dense2 = nn.Linear(16, 1)
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.dropout(out)
        out = nn.ReLU()(self.dense1(out))
        out = self.dense2(out)
        return out

# === 2. تحميل الموديل بشكل صحيح ===
model = LSTMModel()
model.load_state_dict(torch.load("model_v3.pt", map_location='cpu'))
model.eval()

# === 3. تحميل المعاير ===
with open("normalizers.pkl", 'rb') as f:
    normalizers = pickle.load(f)

# === 4. تحميل آخر 7 أيام ===
df = pd.read_csv("pred_result_v3.csv")
df['city'] = df.filter(like='city_').idxmax(axis=1).str.replace('city_', '')

city = "Marrakech"
last_7 = df[df['city'] == city].tail(7).copy()

# === 5. حساب rolling إذا لم يكن موجودًا ===
if 'rad_rolling' not in last_7.columns:
    last_7['rad_rolling'] = last_7['mean_rad'].rolling(3, min_periods=1).mean()
if 'temp_rolling' not in last_7.columns:
    last_7['temp_rolling'] = last_7['temp_max'].rolling(3, min_periods=1).mean()

# === 6. تطبيع ===
def norm(val, feat):
    mean, std = normalizers[city][feat]
    return (val - mean) / std

features = []
for _, row in last_7.iterrows():
    f = [
        norm(row['mean_rad'], 'mean_rad'),
        norm(row['temp_max'], 'temp_max'),
        norm(row['rad_rolling'], 'rad_rolling'),
        norm(row['temp_rolling'], 'temp_rolling')
    ]
    onehot = [1.0 if c == f"city_{city}" else 0.0 for c in normalizers.keys()]
    features.append(f + onehot)

X = torch.tensor([features], dtype=torch.float32)

# === 7. التنبؤ ===
with torch.no_grad():
    prob = torch.sigmoid(model(X)).item()

# === 8. النتيجة ===
tomorrow = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
print(f"احتمال انقطاع الكهرباء ي {city}  ({tomorrow}): غدًا {prob:.1%}")
if prob > 0.6:
    print("تحذير: خطر مرتفع!")
elif prob > 0.4:
    print("تحذير: مراقبة مطلوبة")
else:
    print("الوضع مستقر")

احتمال انقطاع الكهرباء ي Marrakech  (2025-11-16): غدًا 6.2%
الوضع مستقر
